In [1]:
import numpy as np
import csv
import igl
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import geopandas as gpd
from shapely.ops import triangulate
import meshio
import os

In [5]:

# Load the shapefile
current_dir = os.getcwd()
folder_path = os.path.join(current_dir, "ocean")
path = os.path.join(folder_path, "ne_10m_ocean.shp")
shp = gpd.read_file(path)

shp.head()
geometry = shp.geometry.iloc[0]

# check if the geometry is a MultiPolygon
if geometry.geom_type == 'MultiPolygon':
    # if it's a MultiPolygon, extract it as is
    multipolygon = geometry
else:
    # if it's a Polygon, convert it to a MultiPolygon
    multipolygon = geometry.buffer(0)

polygon_xy = []
for ea in geometry.geoms:
    polygon_xy.append(np.array(ea.exterior.coords.xy))

triangles = triangulate(multipolygon)

# convert the triangles to a meshio mesh

mesh = meshio.Mesh(points=polygon_xy, cells=[('triangle', triangles)])

# save the mesh as an OFF file
meshio.write('output.off', mesh, file_format='off')

C:\Users\muhan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\meshio\_mesh.py:129: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.points = np.asarray(points)


ValueError: could not broadcast input array from shape (2,100613) into shape (2,)

In [26]:

# Load the shapefile
current_dir = os.getcwd()
folder_path = os.path.join(current_dir, "ocean")
path = os.path.join(folder_path, "ne_10m_ocean.shp")
sf = shapefile.Reader(path)

# Get the shapes (polygons) from the shapefile
shapes = sf.shapes()

# Initialize the vertices list
vertices = []

# Loop over all shapes
for shape in shapes:
    # Loop over all points in the shape
    for point in shape.points:
        # Add the point to the vertices list
        vertices.append(point)

# Convert the vertices list to a numpy array
vertices = np.array(vertices)

# Open the output OFF file
with open("output.off", "w") as f:
    # Write the header
    f.write("OFF\n")
    f.write("{} 0 0\n".format(len(vertices)))
    # Write the vertices
    for vertex in vertices:
        f.write("{} {} {}\n".format(vertex[0], vertex[1], 0))
    # Write the faces (empty, since the shapefile only contains vertices)
    f.write("0 0 0\n")

# Load the mesh
vertices, faces = igl.read_triangle_mesh("output.off")

# Plot the mesh
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'scene'}, {'type': 'scene'}]])
